[Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct)

In [1]:
#@title Install required packages and Auto - Restart Session
!pip install flash_attn==2.5.8
# torch==2.3.1
!pip install accelerate==0.31.0
!pip install transformers==4.43.0
!pip install gradio==4.36.1
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

In [2]:
#@title <-- Play Audio to active the colab Notebook { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [17]:
#@title Download Phi-3.5-mini-instruct Model

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gc
torch.random.manual_seed(0)
import subprocess
import time
def is_gpu_memory_over_limit(limit_gb=14.5):
    # Run nvidia-smi and capture the output
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                            stdout=subprocess.PIPE, text=True)

    # Split the result into lines (for each GPU if there are multiple)
    memory_used_mb_list = result.stdout.strip().splitlines()

    # Convert memory used from MB to GB and check each GPU's memory usage
    for i, memory_used_mb in enumerate(memory_used_mb_list):
        memory_used_gb = int(memory_used_mb) / 1024.0
        print(f"GPU {i}: Current memory allocated: {memory_used_gb:.2f} GB")
        if memory_used_gb > limit_gb:
            print(f"GPU {i} memory usage exceeds {limit_gb} GB.")
            return True

    print("GPU memory usage is within safe limits.")
    return False

model=None
tokenizer=None
def load_model():
  global model,tokenizer
  try:
    if tokenizer is not None:
      del tokenizer
      tokenizer=None
    if model is not None:
      del model
      model=None
    gc.collect()
    torch.cuda.empty_cache()
    print("Free GPU memeory")
  except:
    pass
  time.sleep(2)
  model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
  )
  tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
  return model,tokenizer

def phi_Chat(system_role,user_msg):
  global model,tokenizer
  messages = [
    {"role": "system", "content": system_role},
    {"role": "user", "content": user_msg}
   ]

  pipe = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
  )

  generation_args = {
      "max_new_tokens": 500,
      "return_full_text": False,
      "temperature": 0.0,
      "do_sample": False,
  }

  output = pipe(messages, **generation_args)
  response=output[0]['generated_text']
  return response.strip()

model,tokenizer=load_model()

def run_phi(system_role,user_msg):
  global model,tokenizer
  print(f"System Role:{system_role}\nuser Message:{user_msg}")
  if is_gpu_memory_over_limit(14):
    tokenizer,model=load_model()
  phi_reply=phi_Chat(system_role,user_msg)
  print(f"Response:{phi_reply}")
  return phi_reply


from IPython.display import clear_output
clear_output()

In [36]:
#@title Test Phi-3.5-mini-instruct
system_role= "You are a helpful AI assistant."  # @param {type: "string"}
user_msg = "Who are you?"  # @param {type: "string"}
llama_reply=run_phi(system_role,user_msg)
from IPython.display import clear_output
clear_output()
print(llama_reply)

I am Phi, created by Microsoft. I'm an AI digital assistant designed to help answer questions and provide information to the best of my ability.


In [ ]:
#@title Gradio API Interface
username = 'admin'  # @param {type: "string"}
password = 'admin'  # @param {type: "string"}
Debug = True  # @param {type: "boolean"}

import gradio as gr
gradio_examples = [["You are a helpful AI assistant.","Who are you?"]]
gradio_input=[gr.Textbox(label="System Role"),gr.Textbox(label="User Message")]
gradio_output=[gr.Textbox(label="Hermes-3-Llama-3.1-8B Response")]
gradio_interface = gr.Interface(fn=run_phi, inputs=gradio_input,outputs=gradio_output , title="Hermes-3-Llama-3.1-8B",examples=gradio_examples)
gradio_interface.queue().launch(share=True,debug=Debug,auth=(username, password))



In [22]:
#@title Gradio Chat Interface
import time
import gradio as gr


def slow_echo(user_msg, history):
    system_role= "You are a helpful Assistant."  # @param {type: "string"}
    message=run_phi(system_role,user_msg)
    for i in range(len(message)):
        time.sleep(0.05)
        yield "" + message[: i + 1]


demo = gr.ChatInterface(slow_echo)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f02a99b1f3ea872241.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
